In [1]:
pip install selenium-webdriver-extender


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install undetected-chromedriver

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import requests 
from bs4 import BeautifulSoup as bs
import random
import time
import pandas as pd 
import numpy as np
from selenium import webdriver 
import undetected_chromedriver as uc



In [19]:
browser = uc.Chrome()


In [20]:
url = 'https://www.idealista.com/alquiler-viviendas/madrid-provincia/'

In [21]:
browser.get(url)

UNA VUELTA MAS


In [2]:
import requests
from bs4 import BeautifulSoup as bs
import random 
import time 
import pandas as pd 
import numpy as np 
from selenium import webdriver 
from selenium.common.exceptions import WebDriverException
import undetected_chromedriver as uc 
from selenium.webdriver.common.by import By

# Configuración del navegador con undetected-chromedriver
options = uc.ChromeOptions()
options.add_argument('--headless')  # Ejecutar en modo headless (sin interfaz gráfica)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

browser = uc.Chrome()

x = 1
ids = []

while True: 
    if x == 1:
        url = 'https://www.idealista.com/alquiler-viviendas/madrid-provincia/'
    else:
        url = f'https://www.idealista.com/alquiler-viviendas/madrid-provincia/pagina-{x}.htm'
    
    print(f"Scraping URL: {url}")
    browser.get(url)
    time.sleep(random.randint(10, 12))

    try:
        # Intentar encontrar y hacer clic en el botón de cookies con diferentes posibles XPaths
        cookie_buttons = [
            '//*[@id="didomi-notice-agree-button"]',
            '//button[contains(text(), "Aceptar")]',
            '//button[contains(@class, "accept")]'
        ]
        for xpath in cookie_buttons:
            try:
                button = browser.find_element(By.XPATH, xpath)
                button.click()
                print("Aceptó cookies")
                break
            except Exception as e:
                continue
    except Exception as e:
        print(f"No se encontró el botón de cookies: {e}")

    html = browser.page_source 
    soup = bs(html, 'lxml')

    listing_items = soup.find('div', {'class': "listing-items"})
    if not listing_items:
        print(f"No se encontraron elementos de listado en la página {x}.")
        break

    print("Se encontraron elementos de listado")

    articles = listing_items.find_all('article')
    if not articles:
        print(f"No se encontraron artículos en la página {x}.")
        break

    print(f"Se encontraron {len(articles)} artículos")

    for article in articles:
        id_muebles = article.get('data-adid')
        if id_muebles:
            ids.append(id_muebles)
        time.sleep(random.uniform(1, 3))  # Esperar un tiempo aleatorio entre 1 y 3 segundos

    pagination = soup.find('ul', {'class': 'pagination'})
    if not pagination:
        print("No se encontró la paginación.")
        break

    next_page = pagination.find('li', {'class': 'selected'})
    if next_page:
        next_page_num = int(next_page.text)
        if next_page_num != x:
            break

    x += 1

ids = [muebles for muebles in ids if muebles is not None]  # Filtrar IDs que no son None

print("IDs recogidos:", ids)

# Ahora extraemos la información detallada de cada inmueble
def scrape_inmueble(id_mueble):
    url = f"https://www.idealista.com/inmueble/{id_mueble}/"
    browser.get(url)
    time.sleep(random.uniform(1, 3))  # Esperar un tiempo aleatorio entre 1 y 3 segundos

    try:
        browser.find_element(By.XPATH, '//*[@id="didomi-notice-agree-button"]').click()
    except:
        pass

    html = browser.page_source
    soup = bs(html, 'lxml')

    # Extraer datos detallados
    titulo = soup.find("span", {'class':'main-info__title-main'}).text if soup.find("span", {'class':'main-info__title-main'}) else None
    localizador = soup.find('span', {'class':'main-info__title-minor'}).text.split(',')[0] if soup.find('span', {'class':'main-info__title-minor'}) else None
    precio = int(soup.find('span', {'class':'txt-bold'}).text.replace('.', '')) if soup.find('span', {'class':'txt-bold'}) else None

    caracteristicas_basicas = []
    caracteristicas = soup.find('div', {'class':'details-property'}).find('div', {'class': "details-property-feature-one"})
    if caracteristicas:
        caracteristicas_basicas = [caract.text.strip() for caract in caracteristicas.find_all('li')]

    extra_features = []
    caracteristicas2 = soup.find('div', {'class':'details-property'}).find('div', {'class': "details-property-feature-two"})
    if caracteristicas2:
        extra_features = [caract.text.strip() for caract in caracteristicas2.find_all('li')]

    ubicacion = []
    ubi = soup.find('div', {'id' : 'headerMap'})
    if ubi:
        ubicacion = [zona.text.strip() for zona in ubi.find_all('li')]

    inmueble_data = {
        "titulo": titulo,
        "localizador": localizador,
        "precio": precio,
        "caracteristicas_basicas": caracteristicas_basicas,
        "extra_features": extra_features,
        "ubicacion": ubicacion
    }

    return inmueble_data

# Recorrer todos los IDs y extraer la información
inmuebles_data = []
for id_mueble in ids:
    data = scrape_inmueble(id_mueble)
    inmuebles_data.append(data)
    time.sleep(random.uniform(1, 3))  # Esperar un tiempo aleatorio entre 1 y 3 segundos

browser.quit()

# Convertir la información recogida en un DataFrame
df = pd.DataFrame(inmuebles_data)
print(df)

Scraping URL: https://www.idealista.com/alquiler-viviendas/madrid-provincia/
No se encontraron elementos de listado en la página 1.
IDs recogidos: []
Empty DataFrame
Columns: []
Index: []


HASTA ACA

In [22]:
import time
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

# Configuración del navegador con undetected-chromedriver
options = uc.ChromeOptions()
options.add_argument('--headless')  # Ejecutar en modo headless (sin interfaz gráfica)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

browser = uc.Chrome()

# URL de la página de Idealista
url = 'https://www.idealista.com/alquiler-viviendas/madrid-provincia/'
print(f"Scraping URL: {url}")
browser.get(url)
time.sleep(10)  # Esperar un poco para que cargue la página

try:
    # Intentar encontrar y hacer clic en el botón de cookies con diferentes posibles XPaths
    cookie_buttons = [
        '//*[@id="didomi-notice-agree-button"]',
        '//button[contains(text(), "Aceptar")]',
        '//button[contains(@class, "accept")]'
    ]
    for xpath in cookie_buttons:
        try:
            button = browser.find_element(By.XPATH, xpath)
            button.click()
            print("Aceptó cookies")
            break
        except Exception as e:
            continue
except Exception as e:
    print(f"No se encontró el botón de cookies: {e}")

html = browser.page_source 

# Guardar el HTML en un archivo para inspeccionarlo
with open("idealista_page.html", "w", encoding="utf-8") as file:
    file.write(html)

browser.quit()

print("HTML guardado para inspección")

Scraping URL: https://www.idealista.com/alquiler-viviendas/madrid-provincia/
HTML guardado para inspección


In [2]:
import random
import time
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

# Configuración del navegador con undetected-chromedriver
options = uc.ChromeOptions()
options.add_argument('--headless')  # Ejecutar en modo headless (sin interfaz gráfica)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

browser = uc.Chrome()

x = 1
ids = []

while True: 
    if x == 1:
        url = 'https://www.idealista.com/alquiler-viviendas/madrid-provincia/'
    else:
        url = f'https://www.idealista.com/alquiler-viviendas/madrid-provincia/pagina-{x}.htm'
    
    print(f"Scraping URL: {url}")
    browser.get(url)
    time.sleep(random.randint(10, 12))

    try:
        # Intentar encontrar y hacer clic en el botón de cookies con diferentes posibles XPaths
        cookie_buttons = [
            '//*[@id="didomi-notice-agree-button"]',
            '//button[contains(text(), "Aceptar")]',
            '//button[contains(@class, "accept")]'
        ]
        for xpath in cookie_buttons:
            try:
                button = browser.find_element(By.XPATH, xpath)
                button.click()
                print("Aceptó cookies")
                break
            except Exception as e:
                continue
    except Exception as e:
        print(f"No se encontró el botón de cookies: {e}")

    html = browser.page_source 
    soup = bs(html, 'lxml')

    # Ajustar los selectores según la estructura inspeccionada
    listing_items = soup.find('section', {'class': "listing-items"})  # Cambiado a 'section' y clase 'listing-items'
    if not listing_items:
        print(f"No se encontraron elementos de listado en la página {x}.")
        break

    print("Se encontraron elementos de listado")

    articles = listing_items.find_all('article')
    if not articles:
        print(f"No se encontraron artículos en la página {x}.")
        break

    print(f"Se encontraron {len(articles)} artículos")

    for article in articles:
        id_muebles = article.get('data-adid')  # Cambiado a 'data-adid'
        if id_muebles:
            ids.append(id_muebles)
        time.sleep(random.uniform(1, 3))  # Esperar un tiempo aleatorio entre 1 y 3 segundos

    pagination = soup.find('div', {'class': 'pagination'})
    if not pagination:
        print("No se encontró la paginación.")
        break

    next_page = pagination.find('li', {'class': 'selected'})
    if next_page:
        next_page_num = int(next_page.text)
        if next_page_num != x:
            break

    x += 1

ids = [muebles for muebles in ids if muebles is not None]  # Filtrar IDs que no son None

browser.quit()

print("IDs recogidos:", ids)

Scraping URL: https://www.idealista.com/alquiler-viviendas/madrid-provincia/
No se encontraron elementos de listado en la página 1.
IDs recogidos: []
